# MedTwin - 2-Agent Medical Assistant System

In [1]:
!pip install streamlit pyngrok --quiet

from pyngrok import ngrok
ngrok.set_auth_token("35byMfscC6rxe2Ey767RO45ryEC_4QjGJC4BPYVzUQBJBksF")  # paste your token here


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.0 MB/s eta 0:00:00


In [2]:
# Kill any old streamlit processes (optional but safe)
!pkill streamlit || echo "no old streamlit"

# Start your app
!streamlit run app.py --server.port 8501 >/dev/null 2>&1 &


no old streamlit


## Install Ollama + LangChain

In [3]:
print("Installing Ollama and LangChain...\n")

!pip install -q colab-xterm
!pip install -q langchain==0.3.7
!pip install -q langchain-community==0.3.7
!pip install -q langchain-ollama==0.2.0
!curl -fsSL https://ollama.com/install.sh | sh

print("\n✓ All packages installed!")

Installing Ollama and LangChain...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requir

## Start Ollama Server

In [4]:
import subprocess
import time

import re
from typing import Dict

# ---------- Root / substring helper ----------
def contains_root(text: str, roots) -> bool:
    """
    Returns True if any root/keyword appears inside the text (case-insensitive).
    Example: root 'diabet' will match 'diabetes' and 'diabetic'.
    """
    text_lower = text.lower()
    return any(root in text_lower for root in roots)

process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✓ Ollama server started!")

✓ Ollama server started!


## Download AI Model

In [5]:
print(" Downloading Llama 3.2 (3-5 minutes first time)...\n")
!ollama pull llama3.2:3b
print("\n✓ Model ready!")




✓ Model ready!


In [6]:
import json
import time
import re
from typing import Dict, List
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:3b", temperature=0.3)
print("✓ FREE Llama 3.2 initialized!")

✓ FREE Llama 3.2 initialized!


## IMPROVED Medical Questions

In [7]:
MEDICAL_QUESTIONS = {
    "diabetes": [
        "What's your fasting blood sugar level?",
        "Have you noticed increased thirst or urination?",
        "Any recent fatigue or blurred vision?"
    ],
    "hypertension": [
        "What's your blood pressure reading?",
        "Any headaches, dizziness, or chest discomfort?",
        "Are you taking your hypertension medications?"
    ],

    "heart_disease": [
    "Are you having any chest pain, tightness, or pressure right now?",
    "Does any chest discomfort get worse when you walk, climb stairs, or exercise?",
    "Do you feel short of breath during simple activities like walking or talking?",
    "Have you noticed your heart beating fast, slow, or irregularly?",
    "Do your legs, feet, or ankles swell by the end of the day?",
    "Do your symptoms improve when you rest?"
],

    "copd": [
        "How's your breathing today (1-10)?",
        "Are you coughing or producing mucus?",
        "Used your inhaler/bronchodilator recently?",
        "Any chest tightness or wheezing?",
        "Exposure to smoke, dust, or pollution today?"
    ],
}


## Agent 1 : IMPROVED Q&A Agent with Smart Memory

In [8]:
import re
import json
from typing import Dict, List, Any


# -------------------------------------------------------------------
# Helper functions
# -------------------------------------------------------------------
def contains_root(text: str, roots: List[str]) -> bool:
    for r in roots:
        if r in text:
            return True
    return False


def parse_llm_output(llm_response: str) -> Dict[str, Dict[str, Any]]:
    try:
        data = json.loads(llm_response)
        return data if isinstance(data, dict) else {}
    except Exception:
        return {}


# -------------------------------------------------------------------
# Q&A-style agent with disambiguation and follow-up questions
# -------------------------------------------------------------------
class SymptomQAAgent:
    def __init__(self, llm):
        self.llm = llm
        self.full_conversation: List[str] = []
        self.extracted_info: Dict[str, Any] = {}
        self.current_question_index: int = 0
        self.condition_type: str | None = None
        self.answers: Dict[str, str] = {}
        self.interview_complete: bool = False
        self.red_flag_detected: bool = False

        # heart vs COPD disambiguation state
        self.awaiting_disambiguation: bool = False
        self.possible_conditions: List[str] = []

    # ------------------------------
    # RULE-BASED EXTRACTION
    # ------------------------------
    def rule_based_extract(self, text: str):
        text_lower = text.lower()

        # Timing
        if re.search(r"\bfor (\d+) (day|week|hour)s?\b", text_lower):
            self.extracted_info["timing"] = text

        if any(p in text_lower for p in [
            "days ago", "day ago", "weeks ago", "week ago",
            "last night", "yesterday", "this morning", "hours ago", "since"
        ]):
            self.extracted_info["timing"] = text

        # Severity
        if re.search(r'(\d{1,2})\s*/\s*10', text_lower) or \
           re.search(r'\b(mild|moderate|severe|unbearable)\b', text_lower):
            self.extracted_info["severity"] = text

        # Glucose
        if re.search(r'\bglucose\b|\bblood sugar\b|\bmg/dl\b|\bmmol\b', text_lower):
            self.extracted_info["glucose"] = text

        # Blood pressure
        if re.search(r'\b(bp|blood pressure)\b|\bmmhg\b|\d+/\d+', text_lower):
            self.extracted_info["blood_pressure"] = text

        # Medication
        if re.search(r'\b(medication|meds|pills|tablets|taking|took|missed|forgot)\b', text_lower):
            self.extracted_info["medication"] = text

        # Breathing
        if any(k in text_lower for k in [
            "shortness of breath", "can't breathe", "cannot breathe", "struggling to breathe",
            "difficulty breathing", "hard to breathe", "breathless", "dyspnea"
        ]) or re.search(r'\bbreath(ing)?\b', text_lower):
            self.extracted_info["breathing_status"] = text

        # Cough / mucus
        if re.search(r'\b(cough|phlegm|mucus|sputum)\b', text_lower):
            self.extracted_info["cough_mucus"] = text

        # Respiratory triggers
        if re.search(r'\b(smok(e|ing)?|dust|pollut(ed|ion)?|fumes?|cigarette)\b', text_lower):
            self.extracted_info["resp_triggers"] = text

        # Red flags
        if any(r in text_lower for r in [
            "can't breathe", "cannot breathe", "struggling to breathe",
            "severe chest pain", "crushing chest pain",
            "blue lips", "blue face", "passed out", "fainted"
        ]):
            self.extracted_info["red_flag"] = text
            self.red_flag_detected = True

    # ------------------------------
    # LLM SYNONYM / INTENT EXTRACTION
    # ------------------------------
    def llm_synonym_extract(self, text: str):
        prompt = (
            "Extract all symptoms, medical concepts, and possible intent from this message.\n"
            "Return JSON ONLY with this structure:\n"
            "{\n"
            "  \"symptom1\": {\n"
            "    \"Canonical\": \"...\",\n"
            "    \"Original\": \"...\",\n"
            "    \"Corrected\": \"...\",\n"
            "    \"Synonyms\": [\"...\"],\n"
            "    \"Intent\": \"...\"\n"
            "  }\n"
            "}\n"
            f"Message: \"{text}\""
        )
        raw = self.llm.invoke(prompt).content.strip()
        llm_data = parse_llm_output(raw)

        for key, data in llm_data.items():
            if isinstance(data, dict):
                self.extracted_info[key] = {
                    "canonical": data.get("Canonical", ""),
                    "original": data.get("Original", ""),
                    "corrected": data.get("Corrected", ""),
                    "synonyms": data.get("Synonyms", []),
                    "intent": data.get("Intent", "")
                }

    def extract_info_from_text(self, text: str):
        self.rule_based_extract(text)
        self.llm_synonym_extract(text)

    # ------------------------------
    # LLM CONDITION GUESS (fallback)
    # ------------------------------
    def llm_condition_guess(self, text: str) -> str | None:
        prompt = (
            "Classify the main condition in this message into exactly ONE of:\n"
            "diabetes, hypertension, heart_disease, copd, none.\n"
            "Return ONLY JSON: {\"condition\": \"...\", \"reason\": \"...\"}\n\n"
            f"Message: \"{text}\""
        )
        raw = self.llm.invoke(prompt).content.strip()

        try:
            data = json.loads(raw)
            cond = data.get("condition", "none")
            allowed = {"diabetes", "hypertension", "heart_disease", "copd"}
            return cond if cond in allowed else None
        except Exception:
            return None

    # ------------------------------
    # CONDITION IDENTIFICATION
    # ------------------------------
    def identify_condition(self, patient_input: str) -> str | None:
        patient_lower = patient_input.lower()

        synonym_parts: List[str] = []
        for key, value in self.extracted_info.items():
            if isinstance(value, dict):
                synonym_parts.append(value.get("canonical", ""))
                synonym_parts.append(value.get("corrected", ""))
                synonym_parts.append(value.get("intent", ""))
                syns = value.get("synonyms", [])
                if isinstance(syns, list):
                    synonym_parts.extend(syns)
                else:
                    synonym_parts.append(str(syns))

        combined_text = (patient_lower + " " + " ".join(synonym_parts)).lower()

        # Diabetes / Hypertension
        has_diabetes = contains_root(combined_text, ["diabet", "glucose", "blood sugar"])
        has_hypertension = contains_root(combined_text, ["blood pressure", "hypertens", "bp"])

        # Heart disease signals
        has_heart = contains_root(
            combined_text,
            [
                "heart", "cardiac",
                "chest pain", "chest tightness", "chest discomfort",
                "angina", "pain when walking", "pain when exercising",
                "shortness of breath on activity",
                "heart rate", "palpitations", "fatigue on exertion",
                "electric pumping"
            ]
        )

        # COPD signals
        has_copd_core = contains_root(
            combined_text,
            ["copd", "chronic obstructive", "emphysema", "chronic bronchitis"]
        )
        has_copd_symptoms = contains_root(combined_text, ["breath"]) and \
                            contains_root(combined_text, ["mucus", "phlegm", "sputum"])

        # NEW: COPD indicator = smoking + breathlessness
        smoking_and_breath = (
            contains_root(combined_text, ["smok", "cigarette"]) and
            contains_root(combined_text, ["breath", "gasp", "out of breath", "short of breath"])
        )

        has_copd = has_copd_core or has_copd_symptoms or smoking_and_breath

        # Priority: diabetes
        if has_diabetes and not (has_hypertension or has_heart or has_copd):
            self.condition_type = "diabetes"
            return self.condition_type

        # Priority: hypertension
        if has_hypertension and not (has_diabetes or has_heart or has_copd):
            self.condition_type = "hypertension"
            return self.condition_type

        # HEART vs COPD ambiguity -> require disambiguation
        if has_heart and has_copd and not (has_diabetes or has_hypertension):
            self.condition_type = None
            self.awaiting_disambiguation = True
            self.possible_conditions = ["heart_disease", "copd"]
            return None

        # Unambiguous heart
        if has_heart and not has_copd:
            self.condition_type = "heart_disease"
            return self.condition_type

        # Unambiguous COPD
        if has_copd and not has_heart:
            self.condition_type = "copd"
            return self.condition_type

        # Fallback: LLM guess
        self.condition_type = self.llm_condition_guess(patient_input)
        return self.condition_type

    # ------------------------------
    # DISAMBIGUATION QUESTION (HEART vs COPD)
    # ------------------------------
    def get_disambiguation_question(self) -> str:
        return (
            "Your symptoms could be related to your heart or to a chronic lung condition.\n"
            "Please answer in your own words, or choose one option:\n\n"
            "A) I feel abnormal heart activity such as electric pumping, weird or fast heart beats, "
            "or a tight band feeling in my upper chest, but I do NOT have heavy mucus or phlegm.\n\n"
            "B) I have a long-lasting cough with thick or heavy mucus or phlegm, especially when I smoke, "
            "or when I am around dust, fumes, or pollution."
        )

    # ------------------------------
    # DISAMBIGUATION ANSWER HANDLER
    # (this is where A/B is turned into heart vs COPD)
    # ------------------------------
    def handle_disambiguation_answer(self, answer: str) -> str | None:
        text = answer.lower()

        # Explicit heart indicators
        heart_indicators = [
            "electric pump", "electric pumping",
            "weird heart beat", "weird heartbeat",
            "fast heart beat", "fast heartbeat",
            "irregular heart beat", "irregular heartbeat",
            "palpitations"
        ]

        # Explicit COPD indicators
        copd_indicators = [
            "heavy mucus", "thick mucus", "a lot of mucus",
            "mucus", "phlegm", "cough with mucus", "coughing mucus"
        ]

        heart_hit = any(ind in text for ind in heart_indicators)
        copd_hit = any(ind in text for ind in copd_indicators)

        if heart_hit and not copd_hit:
            return "heart_disease"
        if copd_hit and not heart_hit:
            return "copd"

        # If user just answers A or B
        if re.search(r"\b(a)\b", text) and not re.search(r"\b(b)\b", text):
            return "heart_disease"
        if re.search(r"\b(b)\b", text) and not re.search(r"\b(a)\b", text):
            return "copd"

        return None

    # ------------------------------
    # QUESTION SELECTION
    # ------------------------------
    def should_skip_question(self, question: str) -> bool:
        q = question.lower()
        if "glucose" in q and "glucose" in self.extracted_info:
            return True
        if "blood pressure" in q and "blood_pressure" in self.extracted_info:
            return True
        if "med" in q and "medication" in self.extracted_info:
            return True
        return False

    def get_next_question(self) -> str | None:
        if not self.condition_type:
            self.interview_complete = True
            return None

        qs = MEDICAL_QUESTIONS.get(self.condition_type, [])
        while self.current_question_index < len(qs):
            q = qs[self.current_question_index]
            if self.should_skip_question(q):
                self.current_question_index += 1
                continue
            return q

        self.interview_complete = True
        return None

    # ------------------------------
    # CHAT (Q&A OUTPUT)
    # ------------------------------
    def chat(self, user_message: str, next_question: str | None = None) -> str:
        self.full_conversation.append(f"Patient: {user_message}")
        # still extract info from every answer
        self.extract_info_from_text(user_message)

        if next_question:
            response_text = next_question
        else:
            response_text = "Thank you. I’ve collected your answers and will analyze them now."

        self.full_conversation.append(f"Agent: {response_text}")
        return response_text

    # ------------------------------
    # INTERVIEW CONTROL
    # ------------------------------
    def start_interview(self, initial_message: str) -> str:
        self.extract_info_from_text(initial_message)
        self.identify_condition(initial_message)

        # Ambiguous heart vs COPD → ask disambiguation Q (A/B)
        if self.awaiting_disambiguation and self.possible_conditions == ["heart_disease", "copd"]:
            question = self.get_disambiguation_question()
            self.full_conversation.append(f"Patient: {initial_message}")
            self.full_conversation.append(f"Agent: {question}")
            return question

        # Unknown condition
        if not self.condition_type:
            self.interview_complete = True
            return "Sorry, I can only assist with diabetes, hypertension, heart disease, or COPD."

        # Normal question flow
        self.current_question_index = 0
        first_q = self.get_next_question()
        return self.chat(initial_message, first_q)

    def continue_interview(self, patient_response: str) -> str:
        # 1) Handle disambiguation answer FIRST (A/B etc.)
        if self.awaiting_disambiguation and self.possible_conditions == ["heart_disease", "copd"]:
            chosen = self.handle_disambiguation_answer(patient_response)

            if not chosen:
                # Fallback to LLM but prefer heart/copd
                guess = self.llm_condition_guess(patient_response)
                if guess in ["heart_disease", "copd"]:
                    chosen = guess
                else:
                    # Last resort default
                    chosen = "heart_disease"

            # Now we have the final condition_type
            self.condition_type = chosen
            self.awaiting_disambiguation = False
            self.current_question_index = 0  # start questions for that category

            next_q = self.get_next_question()
            return self.chat(patient_response, next_q)

        # 2) Normal Q&A flow (save answer to previous question)
        if self.condition_type:
            qs = MEDICAL_QUESTIONS.get(self.condition_type, [])
            if 0 <= self.current_question_index < len(qs):
                prev_q = qs[self.current_question_index]
                self.answers[prev_q] = patient_response

        self.current_question_index += 1
        next_q = self.get_next_question()
        return self.chat(patient_response, next_q)

    # ------------------------------
    # COLLECTED DATA
    # ------------------------------
    def get_collected_data(self) -> Dict[str, Any]:
        return {
            "condition_type": self.condition_type,
            "interview_complete": self.interview_complete,
            "qa_data": {**self.answers, **self.extracted_info},
            "full_conversation": self.full_conversation,
        }


print("✓ SymptomQAAgent (Q&A with heart/COPD disambiguation) is loaded!")


✓ SymptomQAAgent (Q&A with heart/COPD disambiguation) is loaded!


## Agent 2 : Analysis Agent

In [9]:
import json
from typing import Dict, Any


class AnalysisAgent:
    def __init__(self, llm):
        self.llm = llm

    # ✅ Public method (new style)
    def estimate_severity(self, qa_data: Dict) -> str:
        """
        Public method to estimate severity.
        Wrapper around the internal LLM-based method.
        """
        return self._estimate_severity_llm(qa_data)

    # ✅ Backward-compatible alias (for any code that uses .estimateseverity)
    def estimateseverity(self, qa_data: Dict) -> str:
        """
        Alias to keep old code working.
        """
        return self.estimate_severity(qa_data)

    def _estimate_severity_llm(self, qa_data: Dict) -> str:
        """
        Ask the LLM to classify severity into:
        - CRITICAL
        - HIGH
        - MODERATE
        - LOW

        Then enforce some safety rules (e.g., red_flag -> at least CRITICAL).
        """
        data = qa_data.get("qa_data", {})
        full_conv = qa_data.get("full_conversation", [])

        red_flag_present = "red_flag" in data

        prompt = f"""
You are a medical triage assistant (not a doctor).
Your job is to classify the OVERALL severity of this case into ONE of four labels:

- CRITICAL  = life-threatening red-flag symptoms, needs emergency care NOW.
- HIGH      = serious symptoms or unstable control, needs urgent (same day / 24h) medical review.
- MODERATE  = significant but not immediately dangerous, needs medical review within a few days.
- LOW       = mild, stable, routine follow-up is sufficient.

You MUST respond ONLY in JSON with this format:
{{
  "severity_label": "CRITICAL|HIGH|MODERATE|LOW",
  "reason": "short explanation of why you chose this label"
}}

Here is the structured data:

Condition type: {qa_data.get("condition_type", "unknown")}

Conversation:
{chr(10).join(full_conv)}

Collected QA data:
{json.dumps(data, indent=2)}
"""

        try:
            raw = self.llm.invoke(prompt).content.strip()
            parsed = json.loads(raw)
            label = str(parsed.get("severity_label", "")).upper().strip()
        except Exception:
            # Fallback if parsing fails
            label = "MODERATE"

        # Normalize to allowed set
        allowed = {"CRITICAL", "HIGH", "MODERATE", "LOW"}
        if label not in allowed:
            label = "MODERATE"

        # Safety override: if red_flag present, upgrade to CRITICAL if LLM was too soft
        if red_flag_present and label in {"LOW", "MODERATE"}:
            label = "CRITICAL"

        return label

    def analyze(self, qa_data: Dict) -> str:
        # Let LLM be responsible for severity (with our safety override)
        computed_severity = self._estimate_severity_llm(qa_data)

        # Map internal condition_type to a readable disease name
        condition_key = qa_data.get("condition_type", "unknown")
        condition_name_map = {
            "copd": "Chronic Obstructive Pulmonary Disease (COPD)",
            "diabetes": "Diabetes",
            "hypertension": "Hypertension",
            "heart_disease": "Heart Disease",
        }
        condition_name = condition_name_map.get(condition_key, condition_key)

        prompt = f"""You are a medical triage summarizer for a symptom-checking AI.
You are NOT a doctor and cannot diagnose, but you can summarize and triage.

You MUST use the SEVERITY_LABEL exactly as given below.

Condition key: {condition_key}
Condition name: {condition_name}

FINAL TRIAGE SEVERITY (already decided):
- SEVERITY_LABEL: {computed_severity}

Full Conversation:
{chr(10).join(qa_data.get('full_conversation', []))}

Collected Data (questions + extracted info):
{json.dumps(qa_data.get('qa_data', {}), indent=2)}

TASK:
Based on the FINAL TRIAGE SEVERITY and the data above, write a brief analysis:

0. *CONDITION NAME:* Use this exact label: {condition_name}

1. *SEVERITY:* Use EXACTLY this label: {computed_severity}
2. *KEY FINDINGS:* List 2–4 key clinical points (mention any red flags if present, or say 'No red flags reported').
3. *MEDICATION:* Comment briefly on whether they seem compliant or not (based on answers, or say 'Not enough information').
4. *URGENCY:* Suggest a timeframe using the severity (e.g., 'Immediate emergency care', 'Within 24 hours', 'Within a few days', 'Routine follow-up').
5. *RECOMMENDATIONS:* 3–4 short, practical suggestions (self-monitoring, lifestyle, and when to seek urgent help).
   Always remind that this is NOT a diagnosis and they must seek a real doctor.

Be concise, clear, and non-alarming.
Do NOT change the SEVERITY label or the CONDITION NAME.
"""
        response = self.llm.invoke(prompt).content
        return response


print("✓ Analysis Agent ready with LLM-driven severity and safety override!")


✓ Analysis Agent ready with LLM-driven severity and safety override!


## Agent 4 : Planning Agent

In [10]:
class PlanningAgent:
    """
    PlanningAgent:
    --------------
    Takes a condition_type + severity_label and simulates different treatment / lifestyle plans.
    Uses the LLM only to generate a short, patient-friendly explanation for each plan.

    NOTE: This is NOT real medical advice. It's only for educational / simulation purposes.
    """

    def __init__(self, llm):
        self.llm = llm

        # Core library of scenario plans
        # Each plan has:
        #   - name
        #   - base_success (0–1)
        #   - side_effects
        #   - nutrition
        #   - therapy  (education / rehab / counseling)
        #   - sports   (exercise / activity)
        #   - monitoring (what to track at home / clinic)
        self.condition_plans = {
            # ==========================
            # DIABETES (all 4 severities)
            # ==========================
            "diabetes": {
                "CRITICAL": [
                    {
                        "name": "Emergency hospital care + IV insulin + fluids",
                        "base_success": 0.78,
                        "side_effects": "Risk of low blood sugar, electrolyte imbalance, hospital complications.",
                        "nutrition": "Nothing by mouth initially; then gradual re-introduction of diabetic-friendly meals.",
                        "therapy": "Intensive diabetes education once stable; review of injection technique and sick-day rules.",
                        "sports": "No exercise during acute crisis; later, short supervised walking when allowed.",
                        "monitoring": "Continuous glucose checks, blood pressure, urine ketones, and electrolytes."
                    }
                ],
                "HIGH": [
                    {
                        "name": "Basal-bolus insulin + structured diet + close follow-up",
                        "base_success": 0.85,
                        "side_effects": "Risk of hypoglycemia, weight gain if calories not controlled.",
                        "nutrition": "Three main meals and 2–3 snacks, carb counting, avoid sugary drinks.",
                        "therapy": "One-to-one diabetes education, insulin titration plan, psychological support if needed.",
                        "sports": "Gentle daily walking (10–20 min) once glucose is stabilized.",
                        "monitoring": "Home glucose monitoring several times per day, symptom diary, regular clinic visits."
                    }
                ],
                "MODERATE": [
                    {
                        "name": "Oral meds (e.g., metformin) + diet overhaul + regular aerobic exercise",
                        "base_success": 0.92,
                        "side_effects": "Possible GI upset, mild nausea early on.",
                        "nutrition": "Balanced meals with high fiber, low refined sugar, controlled portions.",
                        "therapy": "Group diabetes education, nutrition counseling, weight management plan.",
                        "sports": "30 minutes brisk walking 5x/week, light strength training 2x/week.",
                        "monitoring": "Home glucose checks as advised, weight tracking, blood tests every few months."
                    }
                ],
                "LOW": [
                    {
                        "name": "Lifestyle-focused plan: weight control + healthy diet + light activity",
                        "base_success": 0.97,
                        "side_effects": "Minimal risk if no medications used.",
                        "nutrition": "Mediterranean or DASH-style diet; lots of vegetables, lean proteins, whole grains.",
                        "therapy": "Lifestyle coaching, digital apps to track calories/steps.",
                        "sports": "Daily walking, stretching, yoga or low-impact sports.",
                        "monitoring": "Periodic glucose and A1c checks, self-monitoring of weight and symptoms."
                    }
                ]
            },

            # ==========================
            # HEART DISEASE (all 4)
            # ==========================
            "heart_disease": {
                "CRITICAL": [
                    {
                        "name": "Emergency cardiac care + oxygen + strict bedrest",
                        "base_success": 0.75,
                        "side_effects": "Risk of invasive procedures, bleeding, hospital-acquired complications.",
                        "nutrition": "Cardiac hospital diet; low salt, limited fluids, avoid heavy meals.",
                        "therapy": "Continuous cardiac monitoring, possible interventions (stents, procedures).",
                        "sports": "No exercise; only passive limb movements in bed as allowed.",
                        "monitoring": "Telemetry, frequent vitals, ECGs, lab markers (troponin, etc.)."
                    }
                ],
                "HIGH": [
                    {
                        "name": "Hospital treatment + optimized meds + restricted activity",
                        "base_success": 0.80,
                        "side_effects": "Potential low blood pressure, kidney issues, medication side effects.",
                        "nutrition": "Low-salt, fluid-restricted diet if heart failure present.",
                        "therapy": "Inpatient cardiac education, discharge planning, smoking cessation if relevant.",
                        "sports": "Bed-to-chair transfers only, short supervised standing/walking.",
                        "monitoring": "Daily weight, oxygen saturation, blood pressure, heart rate logs."
                    }
                ],
                "MODERATE": [
                    {
                        "name": "Outpatient meds + cardiac rehab + graded exercise",
                        "base_success": 0.90,
                        "side_effects": "Dizziness, fatigue, possible medication-related side effects.",
                        "nutrition": "Heart-healthy diet: low salt, high vegetables, healthy fats (olive oil, nuts).",
                        "therapy": "Structured cardiac rehab program with supervised sessions.",
                        "sports": "Short walks, light cycling or treadmill as tolerated, no heavy lifting.",
                        "monitoring": "Home BP and pulse diary, symptom log (chest pain, breathlessness, swelling)."
                    }
                ],
                "LOW": [
                    {
                        "name": "Long-term prevention: lifestyle optimization + regular follow-up",
                        "base_success": 0.97,
                        "side_effects": "Minimal; usually related to low-dose meds if any.",
                        "nutrition": "Mediterranean-style diet, limit trans fats and processed foods.",
                        "therapy": "Lifestyle counseling, stress management (breathing, relaxation).",
                        "sports": "Swimming, walking, biking; moderate intensity most days of the week.",
                        "monitoring": "Annual heart check-ups, lipid panel, blood pressure monitoring."
                    }
                ]
            },

            # ==========================
            # COPD (all 4)
            # ==========================
            "copd": {
                "CRITICAL": [
                    {
                        "name": "ICU-level care + oxygen + possible ventilation",
                        "base_success": 0.65,
                        "side_effects": "High risk of infection, delirium, complications from ventilation.",
                        "nutrition": "High-calorie, high-protein small meals to reduce breathing effort.",
                        "therapy": "Intensive respiratory therapy, frequent suctioning, chest physiotherapy.",
                        "sports": "No active sports; only passive limb movements as tolerated.",
                        "monitoring": "Blood gases, oxygen saturation, respiratory rate, signs of fatigue/exhaustion."
                    }
                ],
                "HIGH": [
                    {
                        "name": "Hospital therapy + inhalers + oxygen + minimal activity",
                        "base_success": 0.70,
                        "side_effects": "Possible shakiness, palpitations, dry mouth from inhalers.",
                        "nutrition": "Small frequent meals, avoid gas-forming foods that worsen breathlessness.",
                        "therapy": "Pulmonary physical therapy, breathing techniques training (pursed-lip breathing).",
                        "sports": "Bed-to-chair transfers, very short hallway walks with assistance.",
                        "monitoring": "Oxygen saturation, dyspnea scale, sputum amount/color, temperature."
                    }
                ],
                "MODERATE": [
                    {
                        "name": "Inhaler optimization + pulmonary rehab + gentle exercise",
                        "base_success": 0.82,
                        "side_effects": "Tremor, mild palpitations, dry throat.",
                        "nutrition": "Anti-inflammatory diet with fruits, vegetables, adequate protein.",
                        "therapy": "Scheduled pulmonary rehab sessions and self-management education.",
                        "sports": "Gentle walking, short cycling, avoid cold or polluted air.",
                        "monitoring": "Daily symptom score, inhaler use log, check for flare triggers."
                    }
                ],
                "LOW": [
                    {
                        "name": "Stable COPD management + activity + trigger avoidance",
                        "base_success": 0.90,
                        "side_effects": "Usually mild if inhalers used correctly.",
                        "nutrition": "Balanced diet, keep hydrated, maintain healthy body weight.",
                        "therapy": "Support group, smoking cessation if relevant, regular check-ups.",
                        "sports": "Walking, light swimming, breathing exercises, paced activities.",
                        "monitoring": "Monitor shortness of breath trends, rescue inhaler use, annual spirometry."
                    }
                ]
            },

            # ==========================
            # HYPERTENSION (optional, but kept)
            # ==========================
            "hypertension": {
                "CRITICAL": [
                    {
                        "name": "Hypertensive emergency management in hospital",
                        "base_success": 0.80,
                        "side_effects": "Risk of rapid BP drops, kidney injury, electrolyte shifts.",
                        "nutrition": "Strict low-salt diet, careful fluid control.",
                        "therapy": "Continuous BP monitoring, adjustment of IV and oral medications.",
                        "sports": "No exercise during crisis; gradual mobilization later.",
                        "monitoring": "Frequent BP checks (every minutes to hours), kidney function tests."
                    }
                ],
                "HIGH": [
                    {
                        "name": "Combination antihypertensives + strict low-salt diet",
                        "base_success": 0.85,
                        "side_effects": "Dizziness, electrolyte imbalance, cough with some drugs.",
                        "nutrition": "DASH diet, minimal processed foods, avoid salty snacks & canned foods.",
                        "therapy": "Medication education, adherence support, stress management strategies.",
                        "sports": "Short daily walks, avoid heavy lifting or intense exertion until BP improves.",
                        "monitoring": "Home BP log (morning/evening), regular clinic checks."
                    }
                ],
                "MODERATE": [
                    {
                        "name": "Single agent (e.g., ACE inhibitor) + DASH diet + regular walking",
                        "base_success": 0.88,
                        "side_effects": "Possible cough, mild dizziness when standing quickly.",
                        "nutrition": "High in fruits/vegetables, low-fat dairy, low salt.",
                        "therapy": "Self-monitoring training, phone or app reminders for meds.",
                        "sports": "Daily walking 20–30 min, yoga or stretching.",
                        "monitoring": "Home BP 2–3x/week, keep a simple notebook or phone log."
                    }
                ],
                "LOW": [
                    {
                        "name": "Lifestyle-only approach: diet + exercise + stress control",
                        "base_success": 0.95,
                        "side_effects": "None (if no meds).",
                        "nutrition": "Salt restriction, more potassium-rich foods (fruits, vegetables).",
                        "therapy": "Relaxation training, mindfulness, sleep hygiene.",
                        "sports": "Any enjoyable aerobic activity (walking, cycling, dancing) most days.",
                        "monitoring": "Periodic BP checks at home or pharmacy, annual medical visits."
                    }
                ]
            },


            }

    def compare_plans(self, severity_label: str, condition_type: str, analysis_text: str = None):
        """
        Chooses appropriate plans based on condition_type and severity_label,
        asks the LLM for a short patient-friendly explanation, and prints them.

        severity_label: "CRITICAL" | "HIGH" | "MODERATE" | "LOW"
        condition_type: "diabetes" | "heart_disease" | "hypertension" | "copd" | "asthma"
        analysis_text:  the text coming from AnalysisAgent.analyze(...) (optional)
        """
        print("\n----- Planning Agent Simulation -----")

        condition_type = (condition_type or "").lower().strip()
        severity_label = (severity_label or "").upper().strip()

        # Primary attempt: use the exact severity
        plans = self.condition_plans.get(condition_type, {}).get(severity_label)

        # If no plans found, try mapping CRITICAL -> HIGH as fallback
        if not plans:
            if severity_label == "CRITICAL":
                plans = self.condition_plans.get(condition_type, {}).get("HIGH")

        # If still nothing, we can't simulate a specific plan
        if not plans:
            print("No specific treatment scenarios for this condition/severity. Refer to medical guidelines.")
            return

        for plan in plans:
            base = plan["base_success"]
            prompt = (
                f"Patient summary (from analysis step):\n{analysis_text}\n\n"
                f"Condition: {condition_type}, Severity: {severity_label}\n"
                f"Proposed plan:\n"
                f"  - Name: {plan['name']}\n"
                f"  - Nutrition: {plan['nutrition']}\n"
                f"  - Therapy/Rehab: {plan['therapy']}\n"
                f"  - Exercise/Sports: {plan['sports']}\n"
                f"  - Monitoring: {plan['monitoring']}\n"
                f"  - Predicted Success: {int(base*100)}%\n"
                f"  - Side Effects: {plan['side_effects']}\n\n"
                f"Task: In 4–6 short sentences, explain this plan to the patient "
                f"in simple, kind language. Emphasize that it does NOT replace a real doctor, "
                f"and they must follow their own clinician's advice."
            )

            advice = self.llm.invoke(prompt).content

            print(
                f"\nPlan: {plan['name']}"
                f"\n  Predicted improvement: {int(base*100)}%"
                f"\n  Side effects: {plan['side_effects']}"
                f"\n  Nutrition: {plan['nutrition']}"
                f"\n  Therapy/Rehab: {plan['therapy']}"
                f"\n  Sports/Exercise: {plan['sports']}"
                f"\n  Monitoring: {plan['monitoring']}"
                f"\n  Advice: {advice}\n"
            )


## AGENT 5 : Coordinator Agent (Reports and sending it to DTwin)

In [11]:
from typing import Dict, Any, List

class CoordinatorAgent:
    """
    CoordinatorAgent
    ----------------
    High-level agent that:
      1) Runs AnalysisAgent (triage + severity + summary)
      2) Uses PlanningAgent to build treatment/lifestyle plans
      3) Returns a single structured report for doctors / nurses / caregivers.

    It does NOT replace AnalysisAgent or PlanningAgent – it just orchestrates them.
    """

    def __init__(self, llm):
        self.llm = llm
        self.analysis_agent = AnalysisAgent(llm)
        self.planning_agent = PlanningAgent(llm)

    # 🔧 Helper to get severity label regardless of which method name exists
    def _get_severity_label(self, qa_data: Dict[str, Any]) -> str:
        """
        Try different method names depending on how AnalysisAgent is implemented.
        This makes CoordinatorAgent compatible with multiple versions.
        """
        if hasattr(self.analysis_agent, "estimate_severity"):
            return self.analysis_agent.estimate_severity(qa_data)
        if hasattr(self.analysis_agent, "estimateseverity"):
            return self.analysis_agent.estimateseverity(qa_data)
        if hasattr(self.analysis_agent, "_estimate_severity_llm"):
            return self.analysis_agent._estimate_severity_llm(qa_data)
        # Fallback if something is really wrong
        return "MODERATE"

    def run_full_pipeline(self, qa_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Input:
            qa_data = qaagent.get_collected_data()

        Output:
            A structured report dict:
            {
              "condition_type": ...,
              "severity_label": ...,
              "analysis_text": ...,
              "plans": [ ...list of plan dicts... ],
              "approved_by_clinician": False,
              "timestamp": "TO_FILL_BY_APP"
            }
        """

        # 1) Run analysis (triage + severity)
        analysis_text = self.analysis_agent.analyze(qa_data)
        severity_label = self._get_severity_label(qa_data)
        condition_type = qa_data.get("condition_type", "unknown")

        # 2) Collect treatment / lifestyle plans (without printing)
        plans_data = self._collect_plans(severity_label, condition_type, analysis_text)

        # 3) Build unified report
        report = {
            "condition_type": condition_type,
            "severity_label": severity_label,
            "analysis_text": analysis_text,
            "plans": plans_data,
            "approved_by_clinician": False,   # doctor can toggle this in UI / DB
            "timestamp": "TO_FILL_BY_APP",    # you can fill with datetime later
        }

        return report

    def _collect_plans(self, severity_label: str, condition_type: str, analysis_text: str) -> List[Dict[str, Any]]:
        """
        Internal helper:
        Reuses PlanningAgent.condition_plans but returns structured data instead of printing.
        """

        condition_type_norm = (condition_type or "").lower().strip()
        severity_label_norm = (severity_label or "").upper().strip()

        # Same lookup logic as compare_plans
        plans = self.planning_agent.condition_plans.get(condition_type_norm, {}).get(severity_label_norm)

        # Fallback: CRITICAL -> HIGH if needed
        if not plans and severity_label_norm == "CRITICAL":
            plans = self.planning_agent.condition_plans.get(condition_type_norm, {}).get("HIGH")

        if not plans:
            return []

        results: List[Dict[str, Any]] = []

        for plan in plans:
            base = plan["base_success"]
            prompt = (
                f"Patient summary (from analysis step):\n{analysis_text}\n\n"
                f"Condition: {condition_type_norm}, Severity: {severity_label_norm}\n"
                f"Proposed plan:\n"
                f"  - Name: {plan['name']}\n"
                f"  - Nutrition: {plan['nutrition']}\n"
                f"  - Therapy/Rehab: {plan['therapy']}\n"
                f"  - Exercise/Sports: {plan['sports']}\n"
                f"  - Monitoring: {plan['monitoring']}\n"
                f"  - Predicted Success: {int(base*100)}%\n"
                f"  - Side Effects: {plan['side_effects']}\n\n"
                f"Task: In 4–6 short sentences, explain this plan to the patient "
                f"in simple, kind language. Emphasize that it does NOT replace a real doctor, "
                f"and they must follow their own clinician's advice."
            )

            advice = self.llm.invoke(prompt).content

            results.append({
                "name": plan["name"],
                "predicted_improvement_percent": int(base * 100),
                "side_effects": plan["side_effects"],
                "nutrition": plan["nutrition"],
                "therapy": plan["therapy"],
                "sports": plan["sports"],
                "monitoring": plan["monitoring"],
                "patient_friendly_advice": advice,
            })

        return results

    def send_to_digital_twin(self, report: Dict[str, Any]):
        """
        Placeholder for future integration with your Digital Twin backend.
        For now, just prints a stub message.
        (You said you commented out Digital Twin usage, so this is harmless.)
        """
        print("⚙️ [Coordinator] Digital Twin sync is currently disabled (stub).")
        # Keep your real integration commented out for now.


## UserInterface

In [16]:
# ============================================================
# UPDATED USER INTERFACE (NO print, NO input → Streamlit mode)
# ============================================================

class MedTwinUserInterface:
    """
    This UI no longer prints or asks for input().
    It exposes methods for Streamlit to call:
        - start_interview()
        - next_question()
        - get_report()
    """

    def __init__(self, llm):
        self.llm = llm
        self.qaagent = SymptomQAAgent(llm)
        self.analysisagent = AnalysisAgent(llm)
        self.planningagent = PlanningAgent(llm)
        self.coordinator = CoordinatorAgent(llm)

        # UI state
        self.current_question = None
        self.finished = False
        self.qa_data = None

    # ---------------------------------------------------------
    def start_interview(self, initial_problem: str):
        """Initialize interview and return first question."""
        self.finished = False
        self.qaagent.reset()                     # clear previous interview
        self.qaagent.initial_symptom = initial_problem

        # Identify condition
        condition = self.qaagent.identify_condition(initial_problem)
        self.qaagent.condition_type = condition

        # Get first Q
        first_q = self.qaagent.ask_question(initial_problem)
        self.current_question = first_q
        return first_q

    # ---------------------------------------------------------
    def next_question(self, user_answer: str):
        """Send user answer → get next question OR finish."""
        next_q = self.qaagent.process_answer(user_answer)
        self.current_question = next_q

        if next_q is None:
            self.finished = True
            self.qa_data = self.qaagent.get_collected_data()

        return next_q

    # ---------------------------------------------------------
    def get_final_report(self):
        """Run analysis + planning + assemble final report."""
        if not self.finished:
            return None

        report = self.coordinator.run_full_pipeline(self.qa_data)
        return report


## Start Here

In [29]:
!pip install streamlit pyngrok > /dev/null
!kill $(lsof -t -i:8501) 2> /dev/null
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 >/dev/null 2>&1 &


In [30]:
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(8501, "http")
public_url


<NgrokTunnel: "https://uncatastrophically-unobliterated-mario.ngrok-free.dev" -> "http://localhost:8501">

In [55]:
%%writefile app.py
# ===========================================
# MedTwin – 3-Agent Architecture + Streamlit UI
# ===========================================
import json
import re
from typing import Dict, List, Any

import streamlit as st
from langchain_ollama import ChatOllama


# ============================================================
# Helper functions + Questions
# ============================================================
def contains_root(text: str, roots: List[str]) -> bool:
    for r in roots:
        if r in text:
            return True
    return False


def parse_llm_output(llm_response: str) -> Dict[str, Dict[str, Any]]:
    """Try JSON first, then fall back to empty dict."""
    try:
        data = json.loads(llm_response)
        return data if isinstance(data, dict) else {}
    except Exception:
        return {}


MEDICAL_QUESTIONS: Dict[str, List[str]] = {
    "diabetes": [
        "What's your fasting blood sugar level?",
        "Have you noticed increased thirst or urination?",
        "Any recent fatigue or blurred vision?",
    ],
    "hypertension": [
        "What's your blood pressure reading?",
        "Any headaches, dizziness, or chest discomfort?",
        "Are you taking your hypertension medications?",
    ],
    "heart_disease": [
        "Are you having any chest pain, tightness, or pressure right now?",
        "Does any chest discomfort get worse when you walk, climb stairs, or exercise?",
        "Do you feel short of breath during simple activities like walking or talking?",
        "Have you noticed your heart beating fast, slow, or irregularly?",
        "Do your legs, feet, or ankles swell by the end of the day?",
        "Do your symptoms improve when you rest?",
    ],
    "copd": [
        "How's your breathing today (1–10)?",
        "Are you coughing or producing mucus?",
        "Have you used your inhaler/bronchodilator recently?",
        "Any chest tightness or wheezing?",
        "Have you been around smoke, dust, or pollution today?",
    ],
}


# ============================================================
# SymptomQAAgent – corrected full version
# ============================================================
class SymptomQAAgent:
    def __init__(self, llm):
        self.llm = llm
        self.reset()

    # ---------------- RESET STATE ----------------
    def reset(self):
        self.full_conversation: List[str] = []
        self.extracted_info: Dict[str, Any] = {}
        self.answers: Dict[str, str] = {}

        self.condition_type: str | None = None
        self.current_question_index: int = 0
        self.interview_complete: bool = False

        # Disambiguation state
        self.awaiting_disambiguation: bool = False
        self.possible_conditions: List[str] = []

    # ---------------- RULE-BASED EXTRACTION ----------------
    def rule_based_extract(self, text: str):
        lower = text.lower()

        # timing
        if re.search(r"\bfor (\d+) (day|week|hour)s?\b", lower):
            self.extracted_info["timing"] = text

        # severity (e.g. 7/10 or mild/moderate/severe)
        if re.search(r"(\d+)\s*/\s*10", lower) or any(
            w in lower for w in ["mild", "moderate", "severe", "unbearable"]
        ):
            self.extracted_info["severity"] = text

        # glucose
        if any(k in lower for k in ["glucose", "blood sugar", "mg/dl", "mmol"]):
            self.extracted_info["glucose"] = text

        # blood pressure
        if (
            re.search(r"\d+/\d+", lower)
            or "blood pressure" in lower
            or "bp" in lower
        ):
            self.extracted_info["blood_pressure"] = text

        # mucus / cough
        if any(k in lower for k in ["mucus", "phlegm", "sputum", "cough"]):
            self.extracted_info["cough_mucus"] = text

        # smoking / triggers
        if any(
            k in lower
            for k in ["smoke", "smoking", "cigarette", "dust", "pollution", "fumes"]
        ):
            self.extracted_info["resp_triggers"] = text

        # breathing difficulty
        if any(
            k in lower
            for k in [
                "shortness of breath",
                "can't breathe",
                "cannot breathe",
                "dyspnea",
                "out of breath",
                "gasping",
                "hard to breathe",
            ]
        ):
            self.extracted_info["breathing_status"] = text

        # red flags
        if any(
            k in lower
            for k in [
                "severe chest pain",
                "crushing chest pain",
                "blue lips",
                "blue face",
                "passed out",
                "fainted",
            ]
        ):
            self.extracted_info["red_flag"] = text

    # ---------------- LLM-BASED EXTRACTION (OPTIONAL) ----------------
    def llm_synonym_extract(self, text: str):
        prompt = f"""
Extract key symptoms and medical concepts from this patient message.
Return JSON ONLY, mapping concept name to:
{{
  "Canonical": "...",
  "Original": "...",
  "Synonyms": ["...", "..."]
}}
Message: "{text}"
"""
        try:
            raw = self.llm.invoke(prompt).content.strip()
            data = parse_llm_output(raw)
            for k, v in data.items():
                self.extracted_info[k] = v
        except Exception:
            pass

    def extract_info_from_text(self, text: str):
        self.rule_based_extract(text)
        self.llm_synonym_extract(text)

    # ---------------- LLM CONDITION GUESS (FALLBACK) ----------------
    def llm_condition_guess(self, text: str) -> str | None:
        prompt = f"""
Classify the main condition mentioned in this message into ONE of:
["diabetes", "hypertension", "heart_disease", "copd", "none"].

Return JSON like:
{{"condition": "..."}}

Message: "{text}"
"""
        try:
            raw = self.llm.invoke(prompt).content.strip()
            parsed = json.loads(raw)
            cond = parsed.get("condition", "none")
            if cond in ["diabetes", "hypertension", "heart_disease", "copd"]:
                return cond
        except Exception:
            return None
        return None

    # ---------------- DISAMBIGUATION Q (HEART vs COPD) ----------------
    def get_disambiguation_question(self) -> str:
        return (
            "Your symptoms could be related to your heart or to a chronic lung condition.\n\n"
            "Please answer in your own words, or choose one option:\n\n"
            "A) I feel abnormal heart activity such as electric pumping, weird or fast heart beats, "
            "or a tight band feeling in my upper chest, but I do NOT have heavy mucus or phlegm.\n\n"
            "B) I have a long-lasting cough with thick or heavy mucus or phlegm, especially when I smoke, "
            "or when I am around dust, fumes, or pollution."
        )

    def handle_disambiguation_answer(self, answer: str) -> str | None:
        lower = answer.lower()

        heart_indicators = [
            "electric pump",
            "electric pumping",
            "weird heart beat",
            "weird heartbeat",
            "fast heart beat",
            "fast heartbeat",
            "irregular heart beat",
            "irregular heartbeat",
            "palpitations",
            "tight band",
        ]
        copd_indicators = [
            "heavy mucus",
            "thick mucus",
            "a lot of mucus",
            "phlegm",
            "cough with mucus",
            "coughing mucus",
        ]

        heart_hit = any(t in lower for t in heart_indicators)
        copd_hit = any(t in lower for t in copd_indicators)

        if heart_hit and not copd_hit:
            return "heart_disease"
        if copd_hit and not heart_hit:
            return "copd"

        # A/B direct choice
        if re.search(r"\b(a)\b", lower) and not re.search(r"\b(b)\b", lower):
            return "heart_disease"
        if re.search(r"\b(b)\b", lower) and not re.search(r"\b(a)\b", lower):
            return "copd"

        return None

    # ---------------- CONDITION IDENTIFICATION ----------------
    def identify_condition(self, text: str) -> str | None:
        lower = text.lower()

        # diabetes
        if contains_root(lower, ["diabet", "glucose", "blood sugar"]):
            return "diabetes"

        # hypertension
        if contains_root(lower, ["blood pressure", "hypertens", "bp"]):
            return "hypertension"

        # heart disease
        has_heart = contains_root(
            lower,
            [
                "heart",
                "cardiac",
                "chest pain",
                "chest tightness",
                "angina",
                "pain when walking",
                "pain when exercising",
                "shortness of breath on activity",
                "heart rate",
                "palpitations",
                "fatigue on exertion",
                "electric pumping",
            ],
        )

        # copd
        has_copd_core = contains_root(
            lower, ["copd", "chronic obstructive", "emphysema", "chronic bronchitis"]
        )
        has_copd_symptoms = contains_root(lower, ["breath"]) and contains_root(
            lower, ["mucus", "phlegm", "sputum"]
        )
        smoking_and_breath = contains_root(lower, ["smok", "cigarette"]) and contains_root(
            lower, ["breath", "gasp", "out of breath", "short of breath"]
        )
        has_copd = has_copd_core or has_copd_symptoms or smoking_and_breath

        # ambiguous heart vs COPD
        if has_heart and has_copd:
            self.awaiting_disambiguation = True
            self.possible_conditions = ["heart_disease", "copd"]
            return None

        # unambiguous
        if has_heart:
            return "heart_disease"
        if has_copd:
            return "copd"

        # fallback: let LLM guess
        return self.llm_condition_guess(text)

    # ---------------- QUESTION SKIPPING ----------------
    def should_skip_question(self, question: str) -> bool:
        q = question.lower()
        if "blood sugar" in q and "glucose" in self.extracted_info:
            return True
        if "blood pressure" in q and "blood_pressure" in self.extracted_info:
            return True
        return False

    def get_next_question(self) -> str | None:
        """
        Return the NEXT question to ask without marking interview_complete.
        If there are no more questions, returns None.
        """
        if not self.condition_type:
            return None

        qs = MEDICAL_QUESTIONS.get(self.condition_type, [])

        while self.current_question_index < len(qs):
            q = qs[self.current_question_index]
            if self.should_skip_question(q):
                self.current_question_index += 1
                continue
            return q

        # All questions exhausted
        return None

    # ---------------- INTERVIEW FLOW ----------------
    def start_interview(self, initial_message: str) -> str:
        """
        Called once at the beginning with the free-text symptom description.
        """
        self.reset()
        self.full_conversation.append(f"Patient: {initial_message}")
        self.extract_info_from_text(initial_message)
        cond = self.identify_condition(initial_message)
        self.condition_type = cond

        # ambiguous heart vs COPD
        if self.awaiting_disambiguation and self.possible_conditions == [
            "heart_disease",
            "copd",
        ]:
            question = self.get_disambiguation_question()
            self.full_conversation.append(f"Agent: {question}")
            return question

        # unsupported
        if not self.condition_type:
            self.interview_complete = True
            msg = (
                "Sorry, I can only assist with diabetes, hypertension, "
                "heart disease, or COPD."
            )
            self.full_conversation.append(f"Agent: {msg}")
            return msg

        # normal flow – first structured question
        self.current_question_index = 0
        first_q = self.get_next_question()

        if first_q is None:
            # no structured questions (edge case)
            self.interview_complete = True
            closing = "Thank you. I’ve collected your symptoms and will analyze them now."
            self.full_conversation.append(f"Agent: {closing}")
            return closing

        self.full_conversation.append(f"Agent: {first_q}")
        return first_q

    def continue_interview(self, patient_response: str) -> str:
        """
        Called after each answer. Returns next question or closing message.
        """
        # If we were disambiguating heart vs COPD
        if self.awaiting_disambiguation and self.possible_conditions == [
            "heart_disease",
            "copd",
        ]:
            self.full_conversation.append(f"Patient: {patient_response}")
            self.extract_info_from_text(patient_response)

            chosen = self.handle_disambiguation_answer(patient_response)
            if not chosen:
                guess = self.llm_condition_guess(patient_response)
                if guess in ["heart_disease", "copd"]:
                    chosen = guess
                else:
                    chosen = "heart_disease"

            self.condition_type = chosen
            self.awaiting_disambiguation = False
            self.current_question_index = 0

            next_q = self.get_next_question()
            if next_q is None:
                self.interview_complete = True
                closing = "Thank you. I’ve collected your answers and will analyze them now."
                self.full_conversation.append(f"Agent: {closing}")
                return closing

            self.full_conversation.append(f"Agent: {next_q}")
            return next_q

        # Normal QA flow
        self.full_conversation.append(f"Patient: {patient_response}")
        self.extract_info_from_text(patient_response)

        if self.condition_type:
            qs = MEDICAL_QUESTIONS.get(self.condition_type, [])
            if 0 <= self.current_question_index < len(qs):
                prev_q = qs[self.current_question_index]
                self.answers[prev_q] = patient_response

        # Move index forward and fetch next question
        self.current_question_index += 1
        next_q = self.get_next_question()

        if next_q is None:
            # no more questions → now we really finish
            self.interview_complete = True
            closing = "Thank you. I’ve collected your answers and will analyze them now."
            self.full_conversation.append(f"Agent: {closing}")
            return closing

        self.full_conversation.append(f"Agent: {next_q}")
        return next_q

    # ---------------- EXPORT DATA ----------------
    def get_collected_data(self) -> Dict[str, Any]:
        return {
            "condition_type": self.condition_type,
            "interview_complete": self.interview_complete,
            "qa_data": {**self.answers, **self.extracted_info},
            "full_conversation": self.full_conversation,
        }


# ============================================================
# AnalysisAgent
# ============================================================
class AnalysisAgent:
    def __init__(self, llm):
        self.llm = llm

    def _estimate_severity_llm(self, qa_data: Dict) -> str:
        data = qa_data.get("qa_data", {})
        full_conv = qa_data.get("full_conversation", [])
        red_flag_present = "red_flag" in data

        prompt = f"""
You are a medical triage assistant (not a doctor).
Classify the OVERALL severity into one of:
CRITICAL, HIGH, MODERATE, LOW.

Return ONLY JSON like:
{{
  "severity_label": "CRITICAL|HIGH|MODERATE|LOW",
  "reason": "short reason"
}}

Condition type: {qa_data.get("condition_type", "unknown")}

Conversation:
{chr(10).join(full_conv)}

Collected data:
{json.dumps(data, indent=2)}
"""
        try:
            raw = self.llm.invoke(prompt).content.strip()
            parsed = json.loads(raw)
            label = str(parsed.get("severity_label", "")).upper().strip()
        except Exception:
            label = "MODERATE"

        allowed = {"CRITICAL", "HIGH", "MODERATE", "LOW"}
        if label not in allowed:
            label = "MODERATE"

        if red_flag_present and label in {"LOW", "MODERATE"}:
            label = "CRITICAL"

        return label

    def analyze(self, qa_data: Dict) -> str:
        severity = self._estimate_severity_llm(qa_data)

        cond_key = qa_data.get("condition_type", "unknown")
        condition_name_map = {
            "copd": "Chronic Obstructive Pulmonary Disease (COPD)",
            "diabetes": "Diabetes",
            "hypertension": "Hypertension",
            "heart_disease": "Heart Disease",
        }
        condition_name = condition_name_map.get(cond_key, cond_key)

        prompt = f"""
You are a medical triage summarizer for a symptom-checking AI.
You are NOT a doctor and cannot diagnose, but you can summarize and triage.

Use this condition and severity:

CONDITION NAME: {condition_name}
SEVERITY: {severity}

Conversation:
{chr(10).join(qa_data.get("full_conversation", []))}

Collected QA data:
{json.dumps(qa_data.get("qa_data", {}), indent=2)}

TASK:
Write a brief analysis with these sections:

0. *CONDITION NAME:* (use exactly: {condition_name})
1. *SEVERITY:* (use exactly: {severity})
2. *KEY FINDINGS:* 2–4 bullet points. Mention red flags if present or say 'No red flags reported'.
3. *MEDICATION:* Comment if they seem adherent/non-adherent, or say 'Not enough information'.
4. *URGENCY:* Suggest timeframe: 'Immediate emergency care', 'Within 24 hours', 'Within a few days', or 'Routine follow-up'.
5. *RECOMMENDATIONS:* 3–4 short, practical suggestions (self-monitoring, lifestyle, when to seek urgent help).
   Always remind that this is NOT a diagnosis and they must see a real doctor.

Be concise and non-alarming. Do NOT change CONDITION NAME or SEVERITY label.
"""
        return self.llm.invoke(prompt).content


# ============================================================
# PlanningAgent – care plan
# ============================================================
class PlanningAgent:
    def __init__(self, llm):
        self.llm = llm

    def plan(self, analysis_text: str, qa_data: Dict[str, Any]) -> str:
        prompt = f"""
You are a planning assistant working with a triage summary.

Based on the following analysis and the collected QA data,
create a short, structured care plan with these headings:

1. Immediate Self-Care Steps
2. Short-Term Follow-Up (next days/weeks)
3. Questions to Ask the Doctor
4. Safety Net (when to seek urgent/emergency care)

Keep it concise and patient-friendly.

ANALYSIS:
{analysis_text}

QA DATA:
{json.dumps(qa_data.get("qa_data", {}), indent=2)}
"""
        return self.llm.invoke(prompt).content


# ============================================================
# CoordinatorAgent – runs analysis + planning
# ============================================================
class CoordinatorAgent:
    def __init__(self, llm):
        self.llm = llm
        self.analysis_agent = AnalysisAgent(llm)
        self.planning_agent = PlanningAgent(llm)

    def run_full_pipeline(self, qa_data: Dict[str, Any]) -> Dict[str, Any]:
        analysis_text = self.analysis_agent.analyze(qa_data)
        plan_text = self.planning_agent.plan(analysis_text, qa_data)

        return {
            "condition_type": qa_data.get("condition_type"),
            "analysis": analysis_text,
            "plan": plan_text,
        }


# ============================================================
# MedTwinUserInterface – orchestrates QA + Coordinator
# ============================================================
class MedTwinUserInterface:
    def __init__(self, llm):
        self.llm = llm
        self.qaagent = SymptomQAAgent(llm)
        self.coordinator = CoordinatorAgent(llm)

        self.current_question: str | None = None
        self.finished: bool = False
        self.qa_data: Dict[str, Any] | None = None

    def start_interview(self, initial_problem: str) -> str:
        """Reset and start a new interview."""
        self.qaagent = SymptomQAAgent(self.llm)
        first_q = self.qaagent.start_interview(initial_problem)
        self.current_question = first_q
        self.qa_data = None

        # If agent immediately says "sorry, unsupported", end whole flow
        if self.qaagent.interview_complete and self.qaagent.condition_type is None:
            self.finished = True  # no Q&A, no analysis, no EHR
        else:
            self.finished = False

        return first_q

    def next_question(self, user_answer: str) -> str:
        """Pass answer to QA agent and get next question (or closing)."""
        next_q = self.qaagent.continue_interview(user_answer)
        self.current_question = next_q

        if self.qaagent.interview_complete:
            self.finished = True
            self.qa_data = self.qaagent.get_collected_data()

        return next_q

    def get_final_report(self) -> Dict[str, Any] | None:
        if not self.finished or not self.qa_data:
            return None
        return self.coordinator.run_full_pipeline(self.qa_data)


# ============================================================
# EHR TEMPLATE HELPERS (Template 3)
# ============================================================
def extract_severity_from_analysis(analysis: str) -> str:
    """
    Try to extract the SEVERITY label from the analysis text.
    Falls back to 'UNKNOWN' if not found.
    """
    if not analysis:
        return "UNKNOWN"

    # Example pattern in analysis: 1. *SEVERITY:* CRITICAL
    m = re.search(r"\*SEVERITY:\*\s*([A-Z]+)", analysis)
    if m:
        return m.group(1).strip().upper()

    m2 = re.search(r"SEVERITY[:\-]\s*([A-Za-z ]+)", analysis)
    if m2:
        return m2.group(1).strip().upper()

    return "UNKNOWN"


def build_ehr_summary(report: Dict[str, Any], chat_log: List[tuple[str, str]]) -> str:
    """
    Build an EHR-style summary using Template 3 (compact summary sheet),
    WITHOUT including the detailed care plan.
    """
    analysis = (report.get("analysis") or "").strip()
    cond_key = (report.get("condition_type") or "unknown").lower()

    condition_name_map = {
        "copd": "Chronic Obstructive Pulmonary Disease (COPD)",
        "diabetes": "Diabetes",
        "hypertension": "Hypertension",
        "heart_disease": "Heart Disease",
        "unknown": "Unknown",
    }
    condition_name = condition_name_map.get(cond_key, cond_key.title())

    severity = extract_severity_from_analysis(analysis)

    # First user message in chat_log = main complaint
    main_complaint = "Not clearly described."
    for role, text in chat_log:
        if role.lower() == "you":
            main_complaint = text.strip()
            break

    ehr_text = f"""========================================
EHR SUMMARY SHEET
========================================

PATIENT SUMMARY
----------------------------------------
Condition (suspected):   {condition_name}
Severity:                {severity}
Encounter mode:          Remote symptom interview

CLINICAL SNAPSHOT
----------------------------------------
Main complaint:
- {main_complaint}

Key findings:
{analysis}

NOTES
----------------------------------------
- Detailed care plan is shown separately in the main report.
- This summary is generated from a symptom interview.
- It is AI-generated and is NOT a medical diagnosis.
- A licensed clinician must review and confirm any decisions.
"""
    return ehr_text


# ============================================================
# STREAMLIT UI
# ============================================================
st.set_page_config(page_title="MedTwin – Symptom Q&A", page_icon="🏥")

st.title("🏥 MedTwin – Symptom Q&A & Planning")

st.markdown(
    """
This demo collects information about chronic conditions (Diabetes, Hypertension, Heart Disease, COPD)
and generates a triage-style **analysis** and **care plan**.

> ⚠️ This does **NOT** provide diagnosis or replace a real doctor.
> For emergencies, seek urgent medical care immediately.
"""
)


def reset_interview():
    st.session_state.system = MedTwinUserInterface(st.session_state.llm)
    for key in ["current_q", "chat_log", "ehr_decision", "report", "answer_box"]:
        if key in st.session_state:
            del st.session_state[key]
    try:
        st.rerun()
    except Exception:
        st.experimental_rerun()


# ---------- init session ----------
if "llm" not in st.session_state:
    st.session_state.llm = ChatOllama(model="llama3.2:3b", temperature=0.3)

if "system" not in st.session_state:
    st.session_state.system = MedTwinUserInterface(st.session_state.llm)

system: MedTwinUserInterface = st.session_state.system

if "chat_log" not in st.session_state:
    st.session_state.chat_log: List[tuple[str, str]] = []


# -------------------------------------------
# Step 1 – Initial symptom description
# -------------------------------------------
initial_problem = st.text_area(
    "Describe your symptoms in your own words:",
    height=120,
    placeholder="Example: I quit smoking five years ago, but now I get short of breath when walking...",
)

if st.button("Start Interview"):
    if initial_problem.strip():
        first_q = system.start_interview(initial_problem)
        st.session_state.chat_log = [("You", initial_problem), ("Agent", first_q)]
        st.session_state.current_q = first_q
        if "ehr_decision" in st.session_state:
            del st.session_state["ehr_decision"]
        if "report" in st.session_state:
            del st.session_state["report"]
    else:
        st.warning("Please type your symptoms first.")


# -------------------------------------------
# Step 2 – Q&A loop (only if supported condition)
# -------------------------------------------
if "current_q" in st.session_state and not system.finished:
    st.subheader("Interview")

    for role, text in st.session_state.chat_log:
        st.markdown(f"**{role}:** {text}")

    user_answer = st.text_input("Your answer:", key="answer_box")

    if st.button("Send Answer"):
        if user_answer.strip():
            bot_reply = system.next_question(user_answer)
            st.session_state.chat_log.append(("You", user_answer))
            st.session_state.chat_log.append(("Agent", bot_reply))
            st.session_state.current_q = bot_reply
        else:
            st.warning("Please type an answer before sending.")


# -------------------------------------------
# Step 3 – If finished: either unsupported OR full pipeline
# -------------------------------------------
if system.finished:
    # CASE 1 – unsupported condition (no qa_data)
    if system.qa_data is None:
        st.subheader("Interview ended")

        for role, text in st.session_state.chat_log:
            st.markdown(f"**{role}:** {text}")

        st.info(
            "These symptoms are outside the 4 supported conditions "
            "(diabetes, hypertension, heart disease, COPD). "
            "Please consult a real doctor or start a new interview "
            "for one of the supported conditions."
        )

        if st.button("🔁 Start New Interview"):
            reset_interview()

    # CASE 2 – normal pipeline with analysis + plan + EHR
    else:
        if "report" not in st.session_state:
            st.session_state.report = system.get_final_report()

        report = st.session_state.report

        if report:
            st.subheader("Analysis")
            st.markdown(report["analysis"])

            st.subheader("Care Plan")
            st.markdown(report["plan"])

            st.markdown("---")
            st.markdown(
                "Would you like to view this as a structured **EHR-style clinical report**?"
            )

            if "ehr_decision" not in st.session_state:
                col1, col2 = st.columns(2)
                with col1:
                    if st.button("Yes, show my EHR report"):
                        st.session_state.ehr_decision = "yes"
                with col2:
                    if st.button("No, end interview"):
                        st.session_state.ehr_decision = "no"

            if st.session_state.get("ehr_decision") == "no":
                st.success("Thank you for using MedTwin. Stay safe.")
                if st.button("🔁 Start New Interview"):
                    reset_interview()

            if st.session_state.get("ehr_decision") == "yes":
                ehr_text = build_ehr_summary(report, st.session_state.chat_log)

                st.subheader("📄 EHR-style Clinical Report (Summary Sheet)")
                st.markdown(f"```text\n{ehr_text}\n```")

                if st.button("🔁 Start New Interview"):
                    reset_interview()


Overwriting app.py


In [56]:
!kill $(lsof -t -i:8501) 2> /dev/null
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 >/dev/null 2>&1 &


In [57]:
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(8501, "http")
public_url


<NgrokTunnel: "https://uncatastrophically-unobliterated-mario.ngrok-free.dev" -> "http://localhost:8501">